In [ ]:
import cv2
print(cv2.__version__)

In [1]:
import cv2
#print(cv2.__version__)

img = cv2.imread("C:\img_source\img\himawari.png")
#print(type(img))
#print(img)

cv2.imshow('original', img)
cv2.waitKey()
cv2.destroyAllWindows()

#グレスケに変換
imgGray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

cv2.imshow('Gray Image', imgGray)
cv2.waitKey()
cv2.destroyAllWindows()

################
## ガンマ補正 ##
################

#ガンマ値
gamma = 0.1

# 画素値の最大値
imax = imgGray.max()

# ガンマ補正
imgGamma = imax * (imgGray / imax)**(1/gamma)

cv2.imshow('Gamma Image', imgGamma)
cv2.waitKey()
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

#面積が最大の長方形のインデックスを返す関数
def detect_max_rect(img,contours):
    #最大面積をとるcontoursのインデックス
    max_idx = 0
    
    for i in range(len(contours)):
        if cv2.contourArea(contours[max_idx]) < cv2.contourArea(contours[i],
        window_menseki*0.99 > cv2.contourArea(contours[max_idx])                                                      ):
            max_idx = i
    
    return max_idx


# VideoCapture オブジェクトを取得します
capture = cv2.VideoCapture(0)
window_menseki = 800*600

while(True):
    ret, frame = capture.read()
    # resize the window
    windowsize = (800, 600)
    frame = cv2.resize(frame, windowsize)
    
    imgray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    imblur = cv2.GaussianBlur(imgray, (11, 11), 0)
    ret,thresh = cv2.threshold(imblur, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    im, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    target_contour = contours[detect_max_rect(frame,contours)]

    rect = cv2.minAreaRect(target_contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(frame, [box], 0, (0,255,0), 2)
    
    cv2.imshow('title',thresh)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

In [112]:
import cv2
import numpy as np

#切り出し関数
def crop_rect(img, rect):
    center, size, angle = rect
    center = tuple(map(int, center))  # float -> int
    size = tuple(map(int, size))  # float -> int
    img_w, img_h = img.shape[:2]  # 画像の高さ、幅
    
    #cv2.circle(img,center,5,(0,0,255))

    #rectの高さと幅を取得
    height,width=size
    height,width=int(height),int(width)
    #height+=90
        
    #angleが水平方向の線とのなす角を示すので，＋９０している
    #if angle<0:
    #    angle+=90
    
    #affine行列を用いて元画像を回転させる
    M = cv2.getRotationMatrix2D(center, 0, 1)
    rotated = cv2.warpAffine(img, M, (img_h,img_w))
    cv2.imshow('a', rotated)

    # 切り抜く
    cropped = cv2.getRectSubPix(rotated, (height,width), center)

    return cropped

def get_image(img):
    # cannyで輪郭を抽出する
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (9, 9), 0)
    canny = cv2.Canny(imgBlur, 50, 100)
    
    #モルフォロジー変換の膨張(Dilation)をする
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    dilate = cv2.dilate(canny, kernel, iterations=1)
    
    #二値化する
    #thresh = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #thresh = cv2.GaussianBlur(thresh, (5, 5), 0)
    #ret,thresh = cv2.threshold(thresh,115,255,0)
    #cv2.imshow('thresh', thresh)
    
    #輪郭を抽出して，面積の大きい順にソート
    labels,cnts,hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)  # 抽出した輪郭に近似する直線（？）を探す。
    cnts.sort(key=lambda x:cv2.contourArea(x), reverse=True)  # 面積が大きい順に並べ替える。
    
    #矩形を斜めに抽出したいのでminAreaRect()を使用
    rect = cv2.minAreaRect(cnts[0])
    
    #最大面積部分を切り取る
    croped_img = crop_rect(img,rect)
    
    # ---------- For Debug ------------#
    #上位3つを大きい順に赤のグラデーションで
    #out = np.zeros_like(img)
    #cv2.drawContours(out, [cnts[0]], -1, color=(0,0,255), thickness=-1)
    #cv2.drawContours(out, [cnts[1]], -1, color=(0,0,128), thickness=-1)
    #cv2.drawContours(out, [cnts[2]], -1, color=(0,0,64), thickness=-1)
    
    #print(cnts[0])
    #cv2.imshow('croped_img', croped_img)
    #cv2.imshow('max_red', out)
    #cv2.imshow('canny', dilate)
    # ---------- For Debug ------------#
    
    return croped_img
    
    
# VideoCapture オブジェクトを取得します
capture = cv2.VideoCapture(0)

#windowサイズ
windowsize = (800, 600)
    
#始めに置き換えたい四角形を抽出するためのboolean
convert_img_flag = False
    

while(True):
    frame = capture.read()
    frame = cv2.resize(frame, windowsize)
    print("hoge")
    
    cv2.imshow('title',frame)
    
    
    
    #"Q"を押したらプログラム修了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    #"E"を押したら画面上の一番大きい四角形を抽出
    elif cv2.waitKey(1) & 0xFF == ord('e') & convert_img_flag == False:
        advertise_img = convert_image(frame)
        convert_img_flag = True
    #"A"を押したらもう一度上の操作が可能になる
    elif cv2.waitKey(1) & 0xFF == ord('a') & convert_img_flag == True:
        convert_img_flag = False
        
capture.release()
cv2.destroyAllWindows()

[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89

[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89

[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89

[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89

[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89

[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89]]

 [[238  90]]

 [[237  90]]

 ...

 [[242  90]]

 [[241  89]]

 [[240  89]]]
[[[239  89

In [103]:
#写真のリサイズ

from PIL import Image

img = Image.open("../img/dora3.jpg")
def image_resize(width, image_name):
    resize_image = img.resize((width, int(width * img.size[1] / img.size[0])))  # 画像のリサイズ
    resize_image.save('../img/' + image_name)  # 画像の保存

image_resize(400, 'dora1.jpg')